In [138]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

import matplotlib.pyplot as plt
%matplotlib inline

from itertools import product
from collections import defaultdict
from IPython.display import HTML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
import glob
import requests, re, spacy
nlp = spacy.load('en')

#### i fudged up, I pushed this 50k dataset to github and it truncated it to only 1000 reviews each
#### so i'm pulling these from my local ( "NLP-CAP/aclImdb/Train/neg/*.txt") should i make a new repo?

In [119]:
import os
os.getcwd()

'/Users/hemingway/Desktop/Galvanize/NLP-CAP/aclImdb'

In [127]:
#creating a list of file names to call/open since they are all different txt files in a folder
pos = glob.glob("Train/pos/*.txt") #globglob
neg = glob.glob("Train/neg/*.txt") #glob
pos[0],neg[0]

('Train/pos/4715_9.txt', 'Train/neg/1821_4.txt')

In [107]:
#two pos lists: reviews, and scores (scores labelled in filename)
pos_reviews = []
pos_scores = []
for i in range(len(pos)):
    
    x = open(pos[i],'r')
    content = x.read()
    pos_reviews.append(content)
    
    a = pos[i].split('_')
    b = a[1].split('.txt')
    c = [int(s) for s in b if s.isdigit()]
    pos_scores.append(c[0])

In [108]:
#two neg lists: reviews, and scores
neg_reviews = []
neg_scores = []
for i in range(len(neg)):
    
    x = open(neg[i],'r')
    content = x.read()
    neg_reviews.append(content)
    
    a = neg[i].split('_')
    b = a[1].split('.txt')
    c = [int(s) for s in b if s.isdigit()]
    neg_scores.append(c[0])

In [128]:
def clean_line(line):
    '''
    clean_line
    
    input: line (str) - a line from the script
    return: list of cleaned, lemetized tokens
    
    This does all the NLP preprocessing:
        * lower text
        * split on special characters, while removing annotations
        * remove stop words
        * perform lemmetization
        *** MIGHT REMOVE IMPORTANT WORDS IDK MAN
    '''
    line = line.lower()
    line = [x for x in re.findall(r"[\w()']+", line) if x[0] != "(" and x[-1] != ")"]
    s_stop = set(stopwords.words())
    line = ' '.join([word for word in line if word not in s_stop])
    return [token.lemma_ for token in nlp(line)]

In [144]:
p = []
for i in range(len(pos)):
    p.append(clean_line(pos_reviews[i]))

In [147]:
n = []
for i in range(len(pos)):
    n.append(clean_line(pos_reviews[i]))

In [148]:
#made a DF idk why
Movies = pd.DataFrame({'Pos':p})
Movies['PScore']= pos_scores
Movies['Neg']= n
Movies['NScore']= neg_scores

In [149]:
Movies

,Pos,PScore,Neg,NScore
0,"[movie, get, respect, sure, lot, memorable, qu...",9,"[movie, get, respect, sure, lot, memorable, qu...",4
1,"[bizarre, horror, movie, fill, famous, face, s...",8,"[bizarre, horror, movie, fill, famous, face, s...",1
2,"[solid, unremarkable, film, matthau, einstein,...",7,"[solid, unremarkable, film, matthau, einstein,...",4
3,"[strange, feel, alone, theater, occupy, parent...",8,"[strange, feel, alone, theater, occupy, parent...",1
4,"[probably, already, know, 5, additional, episo...",10,"[probably, already, know, 5, additional, episo...",3
5,"[see, movie, two, grown, child, although, clev...",8,"[see, movie, two, grown, child, although, clev...",3
6,"[use, imdb, br, br, give, hefty, vote, favouri...",10,"[use, imdb, br, br, give, hefty, vote, favouri...",4
7,"[good, film, powerful, message, love, redempti...",10,"[good, film, powerful, message, love, redempti...",2
8,"[make, quartet, trio, continue, quality, early...",10,"[make, quartet, trio, continue, quality, early...",1
9,"[mature, admit, shed, tear, film, mature, resp...",10,"[mature, admit, shed, tear, film, mature, resp...",1


In [150]:
n[1]

['bizarre',
 'horror',
 'movie',
 'fill',
 'famous',
 'face',
 'steal',
 'cristina',
 'raines',
 'tv',
 "'s",
 'flamingo',
 'road',
 'pretty',
 'somewhat',
 'unstable',
 'model',
 'gummy',
 'smile',
 'slated',
 'pay',
 'attempt',
 'suicide',
 'guard',
 'gateway',
 'hell',
 'scene',
 'raine',
 'model',
 'well',
 'capture',
 'mood',
 'music',
 'perfect',
 'deborah',
 'raffin',
 'charming',
 'cristina',
 "'s",
 'pal',
 'raine',
 'move',
 'creepy',
 'brooklyn',
 'heights',
 'brownstone',
 'blind',
 'priest',
 'top',
 'thing',
 'really',
 'start',
 'cook',
 'neighbor',
 'include',
 'fantastically',
 'wicked',
 'burgess',
 'meredith',
 'kinky',
 'couple',
 'sylvia',
 'miles',
 'beverly',
 "d'angelo",
 'diabolical',
 'lot',
 'eli',
 'wallach',
 'great',
 'fun',
 'wily',
 'police',
 'detective',
 'movie',
 'nearly',
 'cross',
 'pollination',
 'rosemary',
 "'s",
 'baby',
 'exorcist',
 'combination',
 'base',
 'good',
 'seller',
 'jeffrey',
 'konvitz',
 'sentinel',
 'entertainingly',
 'spooky',
